<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# FAST-OAD-GA Tutorial

FAST-OAD-GA is an add-on package for [FAST-OAD framework](https://github.com/fast-aircraft-design/FAST-OAD) for performing rapid Overall Aircraft Design in the category General Aviation (GA). The computational core of FAST-OAD being based on the  [OpenMDAO framework](https://openmdao.org/).

## 1. Setting up and analyzing the initial problem

To organize our work, we propose to use two user folders `data/` and `workdir/`. For instance, in `data/` we store a XML file which describes the Beechcraft Duchess. In `workdir/`, we store files generated or modified by FAST-OAD.

In [1]:
import os.path as pth
import os
import openmdao.api as om
from fastoad import api as api_cs25
from fastga.command import api as api_cs23
import logging
from fastoad.gui import VariableViewer
import shutil

# Define relative path
DATA_FOLDER_PATH = 'data'
WORK_FOLDER_PATH = 'workdir'

# Remove work folder
shutil.rmtree(WORK_FOLDER_PATH, ignore_errors=True)

# Define files
CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, 'oad_process.yml')
SOURCE_FILE = pth.join(DATA_FOLDER_PATH, 'beechcraft_76.xml')

# For having log messages on screen
logging.basicConfig(level=logging.INFO, format='%(levelname)-8s: %(message)s')

# For using all screen width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

Failed to import module fastga.models.weight.cg.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.loops.unitary_tests.test_wing_area.py
Failed to import module fastga.models.aerodynamics.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.weight.mass_breakdown.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.geometry.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.load_analysis.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.performances.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.aerodynamics.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.geometry.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.weight.mass_breakdown.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.load_analysis.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.handling_qualitie

After defining a configuration file name, we can ask FAST-GA to generate a default configuration file based on the default OAD model implemented in the framework:

In [2]:
api_cs23.generate_configuration_file(CONFIGURATION_FILE, overwrite=True)

INFO    : Sample configuration written in workdir\oad_process.yml


You can now checkout the generated [configuration file](./workdir/oad_process.toml). In this configuration file, we have specified an input file name 'problem_inputs.xml'. We can ask FAST-OAD to generate the inputs of the default model with the CeRAS parameters as default values:

In [3]:
api_cs25.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

INFO    : Problem inputs written in D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\notebooks\tutorial\workdir\problem_inputs.xml


'D:\\fl.lutz\\FAST\\FAST-OAD\\FAST-GA\\src\\fastga\\notebooks\\tutorial\\workdir\\problem_inputs.xml'

You can now checkout the generated [input file](./workdir/problem_inputs.xml). As shown previously in the user file architecture, the values in this file can be modified by the user and will be considered by FAST-OAD when executing a computational process.

A useful feature that FAST-OAD provides is to list the outputs of the model defined in the configuration file:

In [4]:
api_cs25.list_variables(CONFIGURATION_FILE)

NAME,I/O,DESCRIPTION
data:TLAR:NPAX_design,IN,
data:TLAR:luggage_mass_design,IN,
data:TLAR:range,IN,top-level requirement: design range
data:TLAR:v_approach,IN,
data:TLAR:v_cruise,IN,
data:TLAR:v_limit,IN,
data:TLAR:v_max_sl,IN,
data:aerodynamics:propeller:mach,IN,
data:aerodynamics:propeller:reynolds,IN,
data:geometry:cabin:aisle_width,IN,width of aisles


Another useful feature is to list the modules of the model defined in the configuration file:

In [5]:
api_cs25.list_modules(CONFIGURATION_FILE)

AVAILABLE MODULE IDENTIFIERS,MODULE PATH
fastga.aerodynamics.highspeed.legacy,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\aerodynamics\aerodynamics_high_speed.py
fastga.aerodynamics.lowspeed.legacy,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\aerodynamics\aerodynamics_low_speed.py
fastga.aerodynamics.propeller,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\aerodynamics\components\compute_propeller_aero.py
fastga.geometry.alternate,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\geometry\geometry.py
fastga.geometry.legacy,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\geometry\geometry.py
fastga.handling_qualities.all_handling_qualities,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\handling_qualities\handling_qualities.py
fastga.handling_qualities.static_margin,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\handling_qualities\compute_static_margin.py
fastga.handling_qualities.tail_sizing,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\handling_qualities\tail_sizing\update_tail_areas.py
fastga.loads.legacy,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\load_analysis\loads.py
fastga.loop.mtow,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\weight\mass_breakdown\update_mtow.py


Another useful feature is the [N2 diagram](http://openmdao.org/twodocs/versions/latest/basic_guide/make_n2.html) visualization available in OpenMDAO to see the structure of the model:

In [6]:
N2_FILE = pth.join(WORK_FOLDER_PATH, 'n2.html')
api_cs25.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=N2_FILE, width='100%', height='500px')

INFO    : N2 diagram written in D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\notebooks\tutorial\workdir\n2.html


Alternatively, you can create a [WhatsOpt](https://github.com/OneraHub/WhatsOpt-Doc#whatsopt-documentation) account to generate the XDSM of the problem. If your account is created, you may uncomment next lines and run them (this should take ~ 1 min):

In [ ]:
XDSM_FILE = pth.join(WORK_FOLDER_PATH, 'xdsm.html')
api_cs25.write_xdsm(CONFIGURATION_FILE, XDSM_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=XDSM_FILE, width='100%', height='500px')

## 2. Running your first MDA

### Beechcraft
Here we run an MDA, that is solving the multidisciplinary couplings using the different nested solvers in the model, without running the optimization problem even if it is defined in the configuration file.

In [7]:
eval_problem = api_cs25.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\aerodynamics\components\compute_propeller_aero.py:361: UserWarning: Propeller not working in propulsive mode!
  warnings.warn("Propeller not working in propulsive mode!")
D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\aerodynamics\components\compute_propeller_aero.py:349: UserWarning: Propeller element out of calculated polars: contribution canceled!
  warnings.warn("Propeller element out of calculated polars: contribution canceled!")
c:\users\fl.lutz\anaconda3\envs\fast-ga\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
c:\users\fl.lutz\anaconda3\envs\fast-ga\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarnin


aircraft_sizing


D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\aerodynamics\external\vlm\vlm.py:414: RuntimeWarning: invalid value encountered in double_scalars
  htp_e = cl_htp ** 2 / (math.pi * aspect_ratio * cdi_htp)
D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\aerodynamics\external\vlm\vlm.py:414: RuntimeWarning: invalid value encountered in double_scalars
  htp_e = cl_htp ** 2 / (math.pi * aspect_ratio * cdi_htp)


NL: NLBGS 1 ; 12257205.4 1


NL: NLBGS 2 ; 4233566.23 0.345394084


...

NL: NLBGS 10 ; 54534.086 0.00444914515


NL: NLBGS 11 ; 24750.8906 0.00201929312


NL: NLBGS 12 ; 20325.9706 0.00165828751


NL: NLBGS 13 ; 11849.7979 0.000966761798
NL: NLBGS Converged


INFO    : Computation finished after 338.57 seconds
INFO    : Problem outputs written in D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\notebooks\tutorial\workdir\problem_outputs.xml


Let's save these results. We will use them in the next workbook, that shows some post-processing utilities.

In [8]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
Beechcraft_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_Beechcraft_800nm_mda.xml')
shutil.copy(OUTPUT_FILE, Beechcraft_OUTPUT_FILE)

'workdir\\problem_outputs_Beechcraft_800nm_mda.xml'

The `variable-viewer` provides a way to inspect the content of the XML file. The dropdown lists above the table allow to filter the displayed variable.

In [9]:
api_cs25.variable_viewer(OUTPUT_FILE)

In particular, you may inspect the `data:handling_qualities:static_margin` variable.
You will see that its value is slightly negative, which means that the current aircraft configuration is estimated as unstable. We will see later how we can correct this point.

### Beechcraft for 1000 nm range
Here we run an MDA but we change one of the Top Level Aircraft Requirement (TLAR): the range. We choose a 1000 nm range instead of 800 nm for the Beechcraft. For that we use the `VariableViewer` tool on the input file to change the range (do not forget to save!). Just like this:
![variable_viewer](./img/variable_viewer_change_range.gif)

In [10]:
INPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_inputs.xml')
api_cs25.variable_viewer(INPUT_FILE)

Now that the range has been changed, we run again the MDA.

In [12]:
eval_problem = api_cs25.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\aerodynamics\components\compute_propeller_aero.py:361: UserWarning: Propeller not working in propulsive mode!
  warnings.warn("Propeller not working in propulsive mode!")
D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\aerodynamics\components\compute_propeller_aero.py:349: UserWarning: Propeller element out of calculated polars: contribution canceled!
  warnings.warn("Propeller element out of calculated polars: contribution canceled!")
c:\users\fl.lutz\anaconda3\envs\fast-ga\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
c:\users\fl.lutz\anaconda3\envs\fast-ga\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarnin


aircraft_sizing


NL: NLBGS 1 ; 12304635.3 1


NL: NLBGS 2 ; 4233300.17 0.344041094


NL: NLBGS 3 ; 10884884.5 0.884616587


...

NL: NLBGS 14 ; 6702.98864 0.000544753135


NL: NLBGS 15 ; 3332.58011 0.000270839406


NL: NLBGS 16 ; 1647.1682 0.000133865667


NL: NLBGS 17 ; 779.615998 6.33595373e-05
NL: NLBGS Converged


INFO    : Computation finished after 293.26 seconds
INFO    : Problem outputs written in D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\notebooks\tutorial\workdir\problem_outputs.xml


Let's save again these new results, for post-processing them in next notebook.

In [13]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
Beechcraft_1000nm_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_Beechcraft_1000nm_mda.xml')
shutil.copy(OUTPUT_FILE, Beechcraft_1000nm_OUTPUT_FILE)

'workdir\\problem_outputs_Beechcraft_1000nm_mda.xml'

## 3. Running your first MDO

## Beechcraft 800 nm optimization

As seen earlier, the current aircraft configuration is estimated with a fixed rear length (and thus fuselage length). We will unlock this parameter using a simple optimization problem that will move horizontal tail position in order to minimize the mission consumption. 

The default configuration file defines this optimization problem that aims at:
- minimizing the fuel consumption for the mission (objective),
- with respect to the rear fusellage length (design variables): by extension the horizontal tail to wing distance,
- subject to no additional constraint.

However, running the MDO with the propeller performance estimation will take a very long time. Consequently, and since the propeller won't change from oone MDO loop to another, we decided to use a different .yml file for the probleme definition and reuse the propeller performance map computed in the MDA's hence why the input file will be the output file of a previous MDA.

*(This run should take approximately 20 minutes)*

In [14]:
CONFIGURATION_FILE_MDO = pth.join(WORK_FOLDER_PATH, 'oad_process_mdo.yml')
SOURCE_FILE_MDO = pth.join(WORK_FOLDER_PATH, 'problem_outputs_Beechcraft_800nm_mda.xml')
shutil.copy(pth.join(DATA_FOLDER_PATH, 'fastga_mdo.yml'), CONFIGURATION_FILE_MDO)

'workdir\\oad_process_mdo.yml'

In [15]:
# Set back the inputs from the reference Beechcraft 800 nm
api_cs25.generate_inputs(CONFIGURATION_FILE_MDO, SOURCE_FILE_MDO, overwrite=True)

INFO    : Problem inputs written in D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\notebooks\tutorial\workdir\problem_inputs.xml


'D:\\fl.lutz\\FAST\\FAST-OAD\\FAST-GA\\src\\fastga\\notebooks\\tutorial\\workdir\\problem_inputs.xml'

To visualize and edit the optimization problem definition (present in the configuration file .toml) you can use the `optimization_viewer` tool. If design variables or constraints have active bounds they are yellow whereas they are red if they are violated. Modifiying the `Initial Value` will modify the input file defined in the configuration file .toml whereas `Value` corresponds to the value found in the output file defined in the configuration file (here it is the 800 nm MDA run).

In [16]:
api_cs25.optimization_viewer(CONFIGURATION_FILE_MDO)

In [17]:
optim_problem = api_cs25.optimize_problem(CONFIGURATION_FILE_MDO, overwrite=True)

NL: NLBGS 1 ; 12257205.4 1


NL: NLBGS 2 ; 4233299.79 0.345372346


NL: NLBGS 3 ; 10884883.1 0.888039544


NL: NLBGS 4 ; 8559938.51 0.698359718


NL: NLBGS 5 ; 1092555.38 0.0891357647


...

NL: NLBGS 19 ; 0.150680469 0.0027230348


NL: NLBGS 20 ; 0.111475354 0.00201453625


NL: NLBGS 21 ; 0.0677433411 0.00122422949


NL: NLBGS 22 ; 0.0320001061 0.000578292613
NL: NLBGS Converged
Optimization Complete
-----------------------------------


INFO    : Computation finished after 1019.21 seconds
INFO    : Problem outputs written in D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\notebooks\tutorial\workdir\problem_outputs.xml


Let's save these results:

In [18]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
CeRAS_OPT_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_Beechcraft_800nm_mdo.xml')
shutil.copy(OUTPUT_FILE, CeRAS_OPT_OUTPUT_FILE)

'workdir\\problem_outputs_Beechcraft_800nm_mdo.xml'

The `optimizer_viewer` offers a convenient summary of the optimization result:

In [19]:
api_cs25.optimization_viewer(CONFIGURATION_FILE_MDO)

You can use the `VariableViewer` tool to see the optimization results for all variables of the system by loading the .xml output file:

In [20]:
RESULT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
api_cs25.variable_viewer(RESULT_FILE)